In [1]:
# # Install PyTorch and torchvision
!pip install torch==1.10.0+cu113 torchvision==0.11.1+cu113 torchaudio==0.10.0 -f https://download.pytorch.org/whl/torch_stable.html

# # Install Detectron2
!pip install git+https://github.com/facebookresearch/detectron2.git

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.10.0+cu113 (from versions: 1.11.0, 1.11.0+cpu, 1.11.0+cu102, 1.11.0+cu113, 1.11.0+cu115, 1.11.0+rocm4.3.1, 1.11.0+rocm4.5.2, 1.12.0, 1.12.0+cpu, 1.12.0+cu102, 1.12.0+cu113, 1.12.0+cu116, 1.12.0+rocm5.0, 1.12.0+rocm5.1.1, 1.12.1, 1.12.1+cpu, 1.12.1+cu102, 1.12.1+cu113, 1.12.1+cu116, 1.12.1+rocm5.0, 1.12.1+rocm5.1.1, 1.13.0, 1.13.0+cpu, 1.13.0+cu116, 1.13.0+cu117, 1.13.0+cu117.with.pypi.cudnn, 1.13.0+rocm5.1.1, 1.13.0+rocm5.2, 1.13.1, 1.13.1+cpu, 1.13.1+cu116, 1.13.1+cu117, 1.13.1+cu117.with.pypi.cudnn, 1.13.1+rocm5.1.1, 1.13.1+rocm5.2, 2.0.0, 2.0.0+cpu, 2.0.0+cpu.cxx11.abi, 2.0.0+cu117, 2.0.0+cu117.with.pypi.cudnn, 2.0.0+cu118, 2.0.0+rocm5.3, 2.0.0+rocm5.4.2, 2.0.1, 2.0.1+cpu, 2.0.1+cpu.cxx11.abi, 2.0.1+cu117, 2.0.1+cu117.with.pypi.cudnn, 2.0.1+cu118, 2.0.1+rocm5.3, 2.0.1+rocm5.4.2, 2.1.0, 2.1.0+cpu, 2.1.0+cpu.cxx11.abi, 2.1.0+cu118, 2.1.0+cu121, 2

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyyaml==6.0
# !pip install detectron2 -f
# pip install detectron2 - f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

In [ ]:
# !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu102/torch1.9/index.html

In [2]:
import os
os.mkdir("detr_test_results")

In [3]:
!git clone https://github.com/facebookresearch/detr.git

Cloning into 'detr'...
remote: Enumerating objects: 265, done.
remote: Total 265 (delta 0), reused 0 (delta 0), pack-reused 265 (from 1)
Receiving objects: 100% (265/265), 21.19 MiB | 19.57 MiB/s, done.
Resolving deltas: 100% (120/120), done.


<font size=4>b) Dataset<br>
We will use the Pascal VOC dataset for object detection (pretrained models ares trained on
MSCOCO dataset). Even though the dataset is the same for the two detectors, they use
different formats so you have to process and load differently.
For the Detectron model, you will need to download the dataset to Colab but you don’t need
to prepare the dataset by yourself.
To download the dataset, you can use following code in Colab

In [ ]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCtrainval_11-May-2012.tar
!tar -xvf VOCtrainval_11-May-2012.tar

# change folder name to "datasets/VOC2012" by using the following
!mv VOCdevkit datasets

Convert VOC dataset to COCO format

> To convert the VOC dataset to COCO format, you need to convert the xml annotation files
in VOC into a single json file. There are several important details regarding the data format
conversion as well. We provide a notebook for the data format conversion at
https://colab.research.google.com/drive/10IPTFxi0eu41xI3xS_iZjjkWVRnuBgtA?usp=shar
ing which includes all the details.


In [5]:
# This is to get all the annotation files for data format conversion.
import os
anno_root = "./datasets/VOC2012/Annotations"
xmls = [f for f in os.listdir(anno_root) if f.endswith(".xml")]

with open("anno_path_list.txt", "w") as f:
  for xml in xmls:
    f.write(os.path.join(anno_root, xml)+"\n")

In [6]:
# align the labels in VOC dataset to label index in COCO dataset.
COCO_CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]


TRANSLATE = {
    "airplane": "aeroplane",
    "dining table": "diningtable",
    "motorcycle": "motorbike",
    "potted plant": "pottedplant",
    "couch": "sofa",
    "tv": "tvmonitor",
}


label_path = "coco_labels.txt"

i_empty = 0
with open(label_path, "w") as f:
    for i, label in enumerate(COCO_CLASSES[1:]):
        if label == "N/A":
            label = f"empty{i_empty}"
        if label in TRANSLATE:
            label = TRANSLATE[label]
        f.write(f"{label}\n")

In [7]:
!mkdir VOC_coco_format
!mkdir VOC_coco_format/train2017/
!mkdir VOC_coco_format/val2017/
!mkdir VOC_coco_format/annotations/

In [8]:
# Generate .json file from all the xml annotation files
# modify from https://github.com/yukkyo/voc2coco/blob/master/voc2coco.py
import os
import json
import xml.etree.ElementTree as ET
from typing import Dict, List
from tqdm import tqdm


def get_label2id(labels_path: str) -> Dict[str, int]:
    """id is 1 start"""
    with open(labels_path, 'r') as f:
        labels_str = f.read().split("\n")
    labels_ids = list(range(1, len(labels_str)+1))
    return dict(zip(labels_str, labels_ids))


def get_annpaths(ann_dir_path: str = None,
                 ann_ids_path: str = None,
                 ext: str = '',
                 annpaths_list_path: str = None) -> List[str]:
    # If use annotation paths list
    if annpaths_list_path is not None:
        with open(annpaths_list_path, 'r') as f:
            ann_paths = f.read().split()
        return ann_paths

    # If use annotaion ids list
    ext_with_dot = '.' + ext if ext != '' else ''
    with open(ann_ids_path, 'r') as f:
        ann_ids = f.read().split()
    ann_paths = [os.path.join(ann_dir_path, aid+ext_with_dot) for aid in ann_ids]
    return ann_paths


def get_img_ids_from_ann_paths(ann_paths: List[str]):
    img_ids = [os.path.splitext(os.path.basename(ap))[0] for ap in ann_paths]
    num_ids = list(range(1, len(img_ids)+1))
    return dict(zip(img_ids, num_ids))


def get_image_info(annotation_root, img_id_dict=None):
    global id_set
    path = annotation_root.findtext('path')
    if path is None:
        filename = annotation_root.findtext('filename')
    else:
        filename = os.path.basename(path)
    img_name = os.path.basename(filename)
    img_id = os.path.splitext(img_name)[0]
    if img_id_dict is not None:
        assert img_id in img_id_dict, f"Error: {img_id} is not in img_id_dict !"
        img_id = img_id_dict[img_id]


    size = annotation_root.find('size')
    width = int(size.findtext('width'))
    height = int(size.findtext('height'))

    image_info = {
        'file_name': filename,
        'height': height,
        'width': width,
        'id': img_id
    }
    return image_info


def get_coco_annotation_from_obj(obj, label2id):
    label = obj.findtext('name')
    assert label in label2id, f"Error: {label} is not in label2id !"
    category_id = label2id[label]
    bndbox = obj.find('bndbox')
    xmin = int(float(bndbox.findtext('xmin'))) - 1
    ymin = int(float(bndbox.findtext('ymin'))) - 1
    xmax = int(float(bndbox.findtext('xmax')))
    ymax = int(float(bndbox.findtext('ymax')))
    assert xmax > xmin and ymax > ymin, f"Box size error !: (xmin, ymin, xmax, ymax): {xmin, ymin, xmax, ymax}"
    o_width = xmax - xmin
    o_height = ymax - ymin
    ann = {
        'area': o_width * o_height,
        'iscrowd': 0,
        'bbox': [xmin, ymin, o_width, o_height],
        'category_id': category_id,
        'ignore': 0,
        'segmentation': []  # This script is not for segmentation
    }
    return ann


def convert_xmls_to_cocojson(annotation_paths: List[str],
                             label2id: Dict[str, int],
                             output_jsonpath: str,
                             extract_num_from_imgid: bool = True):
    output_json_dict = {
        "images": [],
        "type": "instances",
        "annotations": [],
        "categories": []
    }
    bnd_id = 1  # START_BOUNDING_BOX_ID, TODO input as args ?
    print('Start converting !')

    if extract_num_from_imgid:
        img_id_dict = get_img_ids_from_ann_paths(ann_paths=annotation_paths)
    else:
        img_id_dict = None

    for a_path in tqdm(annotation_paths):
        # Read annotation xml
        ann_tree = ET.parse(a_path)
        ann_root = ann_tree.getroot()

        img_info = get_image_info(annotation_root=ann_root,
                                  img_id_dict=img_id_dict)
        img_id = img_info['id']
        output_json_dict['images'].append(img_info)

        for obj in ann_root.findall('object'):
            ann = get_coco_annotation_from_obj(obj=obj, label2id=label2id)
            ann.update({'image_id': img_id, 'id': bnd_id})
            output_json_dict['annotations'].append(ann)
            bnd_id = bnd_id + 1

    for label, label_id in label2id.items():
        category_info = {'supercategory': 'none', 'id': label_id, 'name': label}
        output_json_dict['categories'].append(category_info)

    with open(output_jsonpath, 'w') as f:
        output_json = json.dumps(output_json_dict)
        f.write(output_json)


def main():
    label2id = get_label2id(labels_path="coco_labels.txt")
    ann_paths = get_annpaths(
        ann_dir_path=None,
        ann_ids_path=None,
        ext="xml",
        annpaths_list_path="anno_path_list.txt"
    )
    convert_xmls_to_cocojson(
        annotation_paths=ann_paths,
        label2id=label2id,
        output_jsonpath="VOC_coco_format/output.json",
        extract_num_from_imgid=True
    )


if __name__ == '__main__':
    main()


Start converting !


100%|██████████| 17125/17125 [00:02<00:00, 7544.60it/s]


In [9]:
!cp VOC_coco_format/output.json VOC_coco_format/annotations/instances_train2017.json
!cp VOC_coco_format/output.json VOC_coco_format/annotations/instances_val2017.json
!cp ./datasets/VOC2012/JPEGImages/* /content/VOC_coco_format/train2017/
!cp ./datasets/VOC2012/JPEGImages/* /content/VOC_coco_format/val2017/

<font size=5>End of dataset conversion




TRAINING of DETR Model

In [11]:
! python detr/main.py --batch_size 2 \
  --resume https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth \
  --coco_path VOC_coco_format/ \
  --output_dir detr_test_results/ \
  --lr 1e-4 \
  --lr_backbone 1e-5 \
  --epochs 2

Not using distributed mode
git:
  sha: 29901c51d7fe8712168b8d0d64351170bc0f83e0, status: has uncommited changes, branch: main

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=2, lr_drop=200, clip_max_norm=0.1, frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, masks=False, aux_loss=True, set_cost_class=1, set_cost_bbox=5, set_cost_giou=2, mask_loss_coef=1, dice_loss_coef=1, bbox_loss_coef=5, giou_loss_coef=2, eos_coef=0.1, dataset_file='coco', coco_path='VOC_coco_format/', coco_panoptic_path=None, remove_difficult=False, output_dir='detr_test_results/', device='cuda', seed=42, resume='https://dl.fbaipublicfiles.com/detr/detr-r50-e632da11.pth', start_epoch=0, eval=False, num_workers=2, world_size=1, dist_url='env://', distributed=False)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_util

<font size=4>DETR Evaluation

c) Evaluation:
Detectron2 provides a high-level evaluator for Pascal VOC, you can use it simply as:
from detectron2.evaluation import PascalVOCDetectionEvaluator.
Evaluator returns Average Precision based on IoU (intersection-of-union); you only
need to report AP50 which is the primary metric for Pascal VOC.
<br>To evaluate DETR, you can simply run:
! python main.py --batch_size 2 --resume <path to your
checkpoints> --eval --no_aux_loss --coco_path
../VOC_coco_format/

In [13]:
# ! python detr/main.py --batch_size 2 --resume detr_test_results/ --eval
! python detr/main.py --batch_size 2 --resume detr_test_results/checkpoint.pth --eval --no_aux_loss --coco_path VOC_coco_format/

Not using distributed mode
git:
  sha: 29901c51d7fe8712168b8d0d64351170bc0f83e0, status: has uncommited changes, branch: main

Namespace(lr=0.0001, lr_backbone=1e-05, batch_size=2, weight_decay=0.0001, epochs=300, lr_drop=200, clip_max_norm=0.1, frozen_weights=None, backbone='resnet50', dilation=False, position_embedding='sine', enc_layers=6, dec_layers=6, dim_feedforward=2048, hidden_dim=256, dropout=0.1, nheads=8, num_queries=100, pre_norm=False, masks=False, aux_loss=False, set_cost_class=1, set_cost_bbox=5, set_cost_giou=2, mask_loss_coef=1, dice_loss_coef=1, bbox_loss_coef=5, giou_loss_coef=2, eos_coef=0.1, dataset_file='coco', coco_path='VOC_coco_format/', coco_panoptic_path=None, remove_difficult=False, output_dir='', device='cuda', seed=42, resume='detr_test_results/checkpoint.pth', start_epoch=0, eval=True, num_workers=2, world_size=1, dist_url='env://', distributed=False)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pre

DETECTRON example

In [ ]:
# For the DETR model, dataset setup is more complex. DETR only supports COCO 2017
# dataset format, which follows the file structure of:
'''
  - path_to_coco/
  -annotations/
  -- instances_train2017.json
  -- instances_val2017.json
  -train2017/ images for training
  -val2017/
'''

'\n  - path_to_coco/\n  -annotations/\n  -- instances_train2017.json\n  -- instances_val2017.json\n  -train2017/ images for training\n  -val2017/\n'

In [ ]:
# While the VOC dataset format is:
'''
  - VOCdevkit/
  - VOC2012
  -Annotations/ annotation xml files for each image
  -ImageSets/
  -JPEGImages/
  -SegmentationClass
  -SegmentationObject
'''

'\n  - VOCdevkit/\n  - VOC2012\n  -Annotations/ annotation xml files for each image\n  -ImageSets/\n  -JPEGImages/\n  -SegmentationClass\n  -SegmentationObject\n'